# Define class of transfer learning

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import time

import torch
import torch.nn as nn
import torch.optim as optim

import torchvision
from torchvision import datasets, models, transforms

class TransferLearningYOLOv5sNetwork(nn.Module):
    def __init__(self):
        super(TransferLearningYOLOv5sNetwork, self).__init__()

        model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
        
        # load a pre-trained model for the feature extractor
        self.feature_extractor = nn.Sequential(*list(model.children())[:-1]).eval()
        self.fc = nn.Linear(self.classifier[-1].in_features, 7)

        # fix the pre-trained network
        for param in self.feature_extractor.parameters():
            param.requires_grad = False

    def forward(self, images):
        features = self.feature_extractor(images)
        x = torch.flatten(features, 1)
        outputs = self.fc(x)
        return features, outputs

# Transfer learning and traning

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms

# Define transforms for image preprocessing
transforms = transforms.Compose([
    transforms.Resize((1920, 1080)),
    transforms.ToTensor(),
])

# Load dataset
train_dataset = ImageFolder(root='./dataset/train/', transform=transforms)
val_dataset = ImageFolder(root='./dataset/val/', transform=transforms)

# Define dataloaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

# Load pre-trained model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # device object
pretrained_model = TransferLearningYOLOv5sNetwork().to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(pretrained_model.parameters(), lr=0.12)

# Train the model
for epoch in range(100):
    # Train for one epoch
    pretrained_model.train()
    for images, targets in train_loader:
        images, targets = images.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = pretrained_model(images)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        
    # Evaluate on validation set
    pretrained_model.eval()
    with torch.no_grad():
        total_loss = 0
        for images, targets in val_loader:
            images, targets = images.to(device), targets.to(device)
            outputs = pretrained_model(images)
            loss = criterion(outputs, targets)
            total_loss += loss.item() * len(images)
        avg_loss = total_loss / len(val_dataset)
        print(f"Epoch {epoch+1}, Validation loss: {avg_loss:.4f}")
        
torch.save(pretrained_model.to(device).state_dict(), './transfer_learned_weights.pt')

### Define compute_mAP function

In [ ]:
from torchvision.ops import box_iou, box_convert

def compute_mAP(all_boxes, all_labels, all_scores, class_names, iou_threshold=0.5, score_threshold=0.5):
    num_classes = len(class_names)
    ap_per_class = {}
    for class_idx in range(num_classes):
        class_name = class_names[class_idx]
        gt_boxes = []
        gt_labels = []
        det_boxes = []
        det_labels = []
        det_scores = []
        for boxes, labels, scores in zip(all_boxes, all_labels, all_scores):
            mask = labels == class_idx
            gt_boxes.append(boxes[mask])
            gt_labels.append(labels[mask])
            det_boxes.append(boxes[~mask])
            det_labels.append(labels[~mask])
            det_scores.append(scores[~mask])

        gt_boxes = np.concatenate(gt_boxes, axis=0)
        gt_labels = np.concatenate(gt_labels, axis=0)
        det_boxes = np.concatenate(det_boxes, axis=0)
        det_labels = np.concatenate(det_labels, axis=0)
        det_scores = np.concatenate(det_scores, axis=0)

        # Convert boxes to xyxy format
        gt_boxes = box_convert(gt_boxes, in_fmt='xywh', out_fmt='xyxy')
        det_boxes = box_convert(det_boxes, in_fmt='cxcywh', out_fmt='xyxy')

        # Compute iou matrix
        iou_matrix = box_iou(gt_boxes, det_boxes)

        # Apply score and iou threshold
        mask = (iou_matrix > iou_threshold) & (det_scores > score_threshold)
        det_boxes = det_boxes[mask]
        det_labels = det_labels[mask]
        det_scores = det_scores[mask]

        # Compute precision and recall
        num_gt_boxes = gt_boxes.shape[0]
        num_det_boxes = det_boxes.shape[0]
        tp = np.zeros(num_det_boxes, dtype=bool)
        fp = np.zeros(num_det_boxes, dtype=bool)
        for i in range(num_det_boxes):
            overlaps = iou_matrix[:, i]
            gt_idx = overlaps.argmax()
            iou = overlaps[gt_idx]
            if iou >= iou_threshold and gt_labels[gt_idx] == class_idx:
                if not tp[i]:
                    tp[i] = True
                else:
                    fp[i] = True
            else:
                fp[i] = True

        tp_cumsum = np.cumsum(tp)
        fp_cumsum = np.cumsum(fp)
        recall = tp_cumsum / num_gt_boxes
        precision = tp_cumsum / (tp_cumsum + fp_cumsum)

        # Compute AP using 11-point interpolation
        recall_interp = np.linspace(0, 1, 11)
        precision_interp = np.interp(recall_interp, recall, precision)
        ap = np.mean(precision_interp)

        ap_per_class[class_name] = ap

    return ap_per_class

# Validation

In [ ]:
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms

# Load test dataset
test_dataset = ImageFolder(root='./dataset/test/', transform=transforms.Compose([
    transforms.Resize((1920, 1080)),
    transforms.ToTensor(),
]))

# Define dataloader
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=8)

# Load transfer learned model
model = torch.load('./transfer_learned_weights.pt')

# Set model to evaluation mode
model.eval()

# Evaluate on test dataset
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
with torch.no_grad():
    all_boxes = []
    all_labels = []
    all_scores = []
    for images, labels in test_loader:
        images = images.to(device)
        outputs = model(images)
        boxes, labels, scores = model.postprocess(outputs, images.shape[-2:])
        all_boxes.append(boxes.cpu().numpy())
        all_labels.append(labels.cpu().numpy())
        all_scores.append(scores.cpu().numpy())

    # Compute mean average precision (mAP) for each class
    ap_per_class = compute_mAP(all_boxes, all_labels, all_scores, test_dataset.classes)
    mAP = sum(ap_per_class.values()) / len(ap_per_class)
    print(f"Mean Average Precision (mAP): {mAP:.3f}")